In [ ]:
# 단어를 시퀀스로 처리, 시퀀스 모델링
# 자연어처리2에서 https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 경로에서 다운받은
# 파일을 사용

# 단어를 시퀀스로 처리

In [ ]:
# 코렙사용시 아래 주석을 풀고 사용하고 주피터 노트북은 이 셀 다음에 있는 
# 데이터를 가져오는 부분이라고 쓰여있는 셀을 실행 
# 반드시 두 셀중에 하나만 실행

# !rm -r aclImdb
# !curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !tar -xf aclImdb_v1.tar.gz
# !rm -r aclImdb/train/unsup

In [ ]:
# 데이터를 가져오는 부분
import urllib.request as req    
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
filename = 'aclImdb_v1.tar.gz'
with req.urlopen(url) as f:
    with open(filename,'wb') as of:
        of.write(f.read())

In [ ]:
# # 압축풀기
# import tarfile
# with tarfile.open(filename,'r:gz') as tr:
#     tr.extractall()

In [ ]:
# # 데이터를 분할 폴더별로 나누기
# import os, pathlib, shutil, random
# base_dir = pathlib.Path('aclImdb')
# val_dir = base_dir / 'val'
# train_dir = base_dir / 'train'
# for category in ('neg', 'pos'):
#     os.makedirs(val_dir / category)
#     files = os.listdir(train_dir / category)
#     random.Random(1337).shuffle(files)
#     num_val_samples = int(0.2*len(files))
#     val_files = files[-num_val_samples:]
#     for fname in val_files:
#         shutil.move(train_dir / category / fname, val_dir / category / fname)

In [ ]:
# 데이터셋 확보
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val",batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test",batch_size=batch_size
)

In [ ]:
text_only_trains_ds = train_ds.map(lambda x,y : x)

### 정수 시퀀스 데이터셋 준비

In [ ]:
from tensorflow.keras import layers
max_length = 600
max_tokens = 20000
text_vectorization =  layers.TextVectorization(
    max_tokens = max_tokens,
    output_mode='int',
    output_sequence_length=max_length
)
text_vectorization.adapt(text_only_trains_ds)

int_train_ds =  train_ds.map(
    lambda x, y : (text_vectorization(x),y), num_parallel_calls=4
)
int_val_ds = val_ds.map(
    lambda x, y : (text_vectorization(x),y), num_parallel_calls=4
)
int_test_ds = test_ds.map(
    lambda x, y : (text_vectorization(x),y), num_parallel_calls=4
)

### 원-핫 인코딩된 벡터 시퀀스로 시퀀스 모델

In [ ]:
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype = 'int64')
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs,outputs)
model.compile(optimizer='adam',
                    loss = 'binary_crossentropy',
                    metrics=['accuracy']
                   )
model.summary()

### 첫번재 시퀀스 모델 훈련

In [ ]:
callback = [
    keras.callbacks.ModelCheckpoint('one_hot_bidir_lstm.keras', save_best_only=True)
]
model.fit(int_train_ds,validation_data=int_val_ds, epochs=10,callbacks=callback)
print(f"테스트 정확도 : {model.evaluate(int_test_ds)[1]:.3f}")

### 단어임베딩

### 임베딩 층으로 단어 임베딩 학습하기

In [ ]:
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)
inputs = keras.Input(shape=(None,), dtype='int64')
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer='rmsprop',loss = tf.keras.losses.binary_crossentropy, metrics=["acc"])
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_lstm.keras", save_best_only=True)
]
model.fit(int_train_ds,validation_data=int_val_ds, epochs=10, callbacks=callback)

model = keras.models.load_model('embeddings_bidir_lstm.keras')
print(f"테스트 정확도 : {model.evaluate(int_test_ds)[1]:.3f}")

### 마스킹을 활성화한 Embedding 층 사용

In [ ]:
inputs = keras.Input(shape=(None,), dtype='int64')
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer='rmsprop',loss = tf.keras.losses.binary_crossentropy, metrics=["acc"])
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_lstm_width_masking.keras", save_best_only=True)
]
model.fit(int_train_ds,validation_data=int_val_ds, epochs=10, callbacks=callback)

model = keras.models.load_model('embeddings_bidir_lstm_width_masking.keras')
print(f"테스트 정확도 : {model.evaluate(int_test_ds)[1]:.3f}")

In [ ]:
#### 사전 훈련된 단어 임베딩 사용
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip -q glove.6B.zip

import zipfile
url = 'http://nlp.stanford.edu/data/glove.6B.zip'
filename = 'glove.6B.zip'
with req.urlopen(url) as f:
    with open(filename,'wb') as of:
        of.write(f.read())

zipex = zipfile.ZipFile(filename)
zipex.extractall('./')
zipex.close()       

### GloVe 단어 임베딩 파일 파싱

In [ ]:
import numpy as np
path_to_glove_file = './glove/glove.6B.100d.txt'
embeddings_index = {}
with open(path_to_glove_file,encoding='utf8') as f:
    for line in f:
        word,coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep= ' ')
        embeddings_index[word] = coefs

print(f"단어 벡터 개수: {len(embeddings_index)}")

### Glove 단어 임베딩 행렬 준비

In [ ]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embdding_matrix = np.zeros((max_tokens, embedding_dim))

In [ ]:
for word, i in  word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embdding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embdding_matrix),
    trainable =False,
    mask_zero =True
)

In [ ]:
inputs = keras.Input(shape=(None,), dtype='int64')
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer='rmsprop',loss = tf.keras.losses.binary_crossentropy, metrics=["acc"])
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras", save_best_only=True)
]
model.fit(int_train_ds,validation_data=int_val_ds, epochs=10, callbacks=callback)

model = keras.models.load_model('glove_embeddings_sequence_model.keras')
print(f"테스트 정확도 : {model.evaluate(int_test_ds)[1]:.3f}")